In [ ]:
from caret_analyze import Application, Architecture, Lttng

TRACE_DATA = '/home/atsushi/2024_RTSS_WiP_Evaluation/autoware_execution_times/session-20240807132325'

lttng = Lttng(TRACE_DATA)
arch = Architecture('lttng', TRACE_DATA)
app = Application(arch, lttng)

In [ ]:
import re
from caret_analyze.runtime import CallbackBase


def filter_cbs(cbs: list[CallbackBase]) -> list[CallbackBase]:
    ignore_node_list = [
        ".*/default_ad_api/.*",
        ".*radar.*",
        ".*_evaluator",
        ".*traffic_light.*",
        "/system/.*",
        "/map/.*",
        "/control/external_.*",
        "/localization/localization_error_monitor",
        "/sensing/imu/gyro_bias_validator",
        "/control/autonomous_emergency_braking",
        "/localization/pose_twist_fusion_filter/pose_instability_detector",
        "/vehicle/calibration/accel_brake_map_calibrator",
        "/pacmod_diag_publisher",
        "/pacmod/socket_can_sender",
        "/pacmod_dynamic_parameter_changer",
        "/pacmod/socket_can_receiver",
        "/pacmod/pacmod",
        "/control/control_validator",
        "/control/autoware_operation_mode_transition_manager",
        "/planning/mission_planning/goal_pose_visualizer",
        "/planning/mission_planning/mission_planner",
        "/planning/mission_planning/route_selector",
        "/planning/scenario_planning/parking/freespace_planner",
        "/planning/scenario_planning/parking/costmap_generator",
    ]

    filtered_cbs = []

    for cb in cbs:
        ignore = False
        # ignore_node_listの各パターンと照合
        for pattern in ignore_node_list:
            if re.match(pattern, cb.callback_name):
                ignore = True
                break

        if not ignore:
            filtered_cbs.append(cb)

    return filtered_cbs

In [ ]:
from caret_analyze.record import Latency
from caret_analyze.runtime import CallbackBase
import pandas as pd
import numpy as np

OUTPUT_DIR = "/home/atsushi/2024_RTSS_WiP_Evaluation/autoware_execution_times"


def cb_to_latency_df(cb: CallbackBase) -> pd.DataFrame:
    return Latency(cb.to_records()).to_records().to_dataframe()


for cb in filter_cbs(app.callbacks):
    latency_df = cb_to_latency_df(cb)
    if latency_df.empty:
        continue
    output_file = f"{OUTPUT_DIR}/{cb.callback_name.replace('/', '-')}.txt"
    np.savetxt(output_file, latency_df.iloc[:, 1].values, fmt='%d')